In [1]:
from bs4 import BeautifulSoup as bs

import warnings
warnings.filterwarnings('ignore')

import requests
import pandas as pd
import numpy as np
import datetime as dt
import os
from scipy.stats import norm as norm

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\DataCollection')
from yahoo_query import *
pd.options.display.float_format = '{:,.6f}'.format
%matplotlib inline
os.chdir('D:\Options Data\CBOE Manual Downloads')

In [74]:
filename = 'quotedata.dat'

vix = yahoo_query('^VIX',dt.datetime.today())
vix.minute_query()
vix = vix.minute_prices.dropna()

options_chain = pd.read_csv(filename, sep=',', header=0, skiprows=2)

i = 0
for line in open(filename,'r'):
    curr_line = [x.strip() for x in line.split(',')]
    if i == 0:
        curr_spx = float(curr_line[1])
    
    if i == 1:
        curr_time = curr_line[0].replace('@','').replace('  ',' ').replace(' ET','')
        curr_time = dt.datetime.strptime(curr_time, '%b %d %Y %H:%M')
    if i == 1:
        break
    i += 1
    
options_chain['Expiration Date'] = pd.to_datetime(options_chain['Expiration Date'])
options_chain = options_chain[options_chain['Expiration Date'] > dt.datetime.today()]

del options_chain['Last Sale'], options_chain['Net']

universal_columns = ['Expiration Date','Strike']
call_columns = list(filter(lambda x: x in universal_columns or '1' not in x, options_chain.columns.tolist()))
put_columns = list(filter(lambda x: x in universal_columns or '1' in x, options_chain.columns.tolist()))

calls = options_chain[call_columns]
del calls['Calls'], calls['Puts']

puts = options_chain[put_columns]
puts.columns = [x.replace('.1','') for x in puts.columns.tolist()]

calls = calls[(calls.IV > 0) & (calls.Bid > 0.05)].reset_index(drop = True).sort_values(['Expiration Date', 'Strike'])
puts = puts[(puts.IV > 0) & (puts.Bid > 0.05)].reset_index(drop = True).sort_values(['Expiration Date', 'Strike'])

calls['Mid'] = (calls['Bid'] + calls['Ask'])/2
puts['Mid'] = (puts['Bid'] + puts['Ask'])/2

calls['T'] = ((calls['Expiration Date'] - dt.datetime(2019,2,15)).dt.days)/365.25
puts['T'] = ((puts['Expiration Date'] - dt.datetime(2019,2,15)).dt.days)/365.25

In [25]:
calls = options_chain[call_columns]
del calls['Calls'], calls['Puts']

a = (calls['IV']**2)/2
b = -calls['IV']*norm.ppf(calls['Delta'])
c = np.log(curr_spx/calls['Strike'])

calls['time_remaining'] = ((-b + np.sqrt(b**2 - 4*a*c))/(2*a))**2

puts = options_chain[put_columns]
puts.columns = [x.replace('.1','') for x in puts.columns.tolist()]

commissions = 0.0266

def create_spreads(spreads, contract_type = 'puts', net_delta_threshold = 0.275):

    spreads['Net Delta'] = abs(spreads['Short Delta'] - spreads['Long Delta'])
    spreads = spreads[spreads['Net Delta'] <= net_delta_threshold]
    spreads['Credit'] = spreads['Short Bid'] - spreads['Long Ask'] - commissions
    spreads['Required_Contracts'] = round(0.5/spreads['Net Delta'])
    spreads['Total Premo'] = spreads.Credit*100*spreads.Required_Contracts
    
    if contract_type == 'puts':
        spreads['Max Loss'] = (spreads['Long Strike'] - spreads['Short Strike'])*100*spreads.Required_Contracts + spreads['Total Premo']
        spreads['Break Even'] = spreads['Short Strike'] - spreads.Credit

        spreads['SPX Equiv Lower Loss'] = (spreads['Long Strike'] - curr_spx)*100
        spreads['SPX Equiv Break Even Loss'] = (spreads['Break Even'] - curr_spx)*100
        spreads['SPX Equiv Gain Level'] = curr_spx + spreads['Total Premo']/100

        spreads['Prob of Better Than ES'] = norm.cdf(spreads['SPX Equiv Gain Level']/curr_spx - 1,0,spreads['Weekly IV']) - norm.cdf(spreads['Break Even']/curr_spx - 1,0,spreads['Weekly IV']) + norm.cdf(spreads['Long Strike']/curr_spx - 1,0,spreads['Weekly IV'])

    if contract_type == 'calls':
        spreads['Max Loss'] = (spreads['Short Strike'] - spreads['Long Strike'])*100*spreads.Required_Contracts + spreads['Total Premo']
        spreads['Break Even'] = spreads['Short Strike'] + spreads.Credit

        spreads['SPX Equiv Lower Loss'] = (curr_spx - spreads['Long Strike'])*100
        spreads['SPX Equiv Break Even Loss'] = -(spreads['Break Even'] - curr_spx)*100
        spreads['SPX Equiv Gain Level'] = curr_spx - spreads['Total Premo']/100

        spreads['Prob of Better Than ES'] = norm.cdf(spreads['Break Even']/curr_spx - 1,0,spreads['Weekly IV']) - norm.cdf(spreads['SPX Equiv Gain Level']/curr_spx - 1,0,spreads['Weekly IV']) + 1 - norm.cdf(spreads['Long Strike']/curr_spx - 1,0,spreads['Weekly IV'])
    
    spreads['EV'] = np.nan
    spreads['Win Prob'] = np.nan
    stepsize = 0.05
    
    for idx, row in spreads.iterrows():
        
        if contract_type == 'puts':
            ev_df = pd.DataFrame({'SPX': np.arange(row['Long Strike'], row['Short Strike'] + stepsize, stepsize)})
            ev_df = ev_df[(ev_df['SPX'] < row['Short Strike'] + stepsize)]
            ev_df['PnL'] = ev_df['SPX'] - row['Short Strike'] + row.Credit
            
        if contract_type == 'calls':
            ev_df = pd.DataFrame({'SPX': np.arange(row['Short Strike'], row['Long Strike'] + stepsize, stepsize)})
            ev_df = ev_df[(ev_df['SPX'] < row['Long Strike'] + stepsize)]
            ev_df['PnL'] = row['Short Strike'] - ev_df['SPX'] + row.Credit
    
        ev_df['Prob'] = norm.cdf(ev_df['SPX']/curr_spx - 1,0,row['Weekly IV'])
        lb = ev_df.loc[0,'Prob']
        ub = ev_df.loc[len(ev_df) - 1,'Prob']

        ev_df['Prob'] = ev_df['Prob'].diff()
        ev_df.loc[0,'Prob'] = lb
        ev_df.loc[len(ev_df) - 1, 'Prob'] = 1 - ub

        ev_df['EV'] = ev_df.Prob*ev_df.PnL

        total_ev = sum(ev_df['EV'])
        win_prob = sum(ev_df[ev_df['PnL'] >= 0]['Prob'])

        spreads.loc[idx, 'EV'] = total_ev
        spreads.loc[idx, 'Win Prob'] = win_prob

    
    return spreads


day_diff = 8

bid_threshold = 0.5

net_delta_threshold = 0.5

weekly_puts = puts[(puts['Expiration Date'] - dt.datetime.today()).dt.days == day_diff]
weekly_puts = weekly_puts[(weekly_puts.Strike <= curr_spx) & 
                          (weekly_puts['Bid'] >= bid_threshold)].sort_values('Strike', ascending = False)
weekly_puts = weekly_puts.drop_duplicates(subset = ['Strike'], keep = 'last').reset_index(drop = True)

weekly_calls = calls[(calls['Expiration Date'] - dt.datetime.today()).dt.days == day_diff]
weekly_calls = weekly_calls[(weekly_calls.Strike >= curr_spx) & 
                            (weekly_calls['Bid'] >= bid_threshold)].sort_values('Strike', ascending = True)
weekly_calls = weekly_calls.drop_duplicates(subset = ['Strike'], keep = 'last').reset_index(drop = True)

time_remaining = weekly_calls.time_remaining.mean()

put_spreads_list = []
for idx, row in weekly_puts.iterrows():
    put_spreads = weekly_puts[weekly_puts.Strike < row.Strike][['Expiration Date','Strike','Ask','Delta']]
    if len(put_spreads) < 3:
        continue
    else:
        put_spreads.columns = ['Expiration Date','Long Strike','Long Ask', 'Long Delta']
        put_spreads['Short Strike'] = row.Strike
        put_spreads['Short Bid'] = row.Bid
        put_spreads['Short Delta'] = row.Delta
        put_spreads['Weekly IV'] = (vix.reset_index().loc[len(vix) - 1, '^VIX_close']/100)*np.sqrt(time_remaining)
    put_spreads_list.append(create_spreads(put_spreads, 'puts', net_delta_threshold))
    
put_spreads = pd.concat(put_spreads_list, axis = 0).dropna().reset_index(drop = True)
put_spreads = put_spreads[(put_spreads['Total Premo'] > weekly_puts.loc[0,'Bid']*100) & 
                          (put_spreads['Prob of Better Than ES'] > 0.5) & 
                          (put_spreads['Max Loss'] > -5000)].reset_index(drop = True)
put_spreads['Max Loss Differences'] = put_spreads['Max Loss'] - put_spreads['SPX Equiv Lower Loss']

put_spreads = put_spreads.sort_values('EV', ascending = False).reset_index(drop = True)
put_spreads[['Expiration Date','Long Strike','Short Strike','Required_Contracts','Credit','Total Premo','Max Loss','EV','Win Prob']]

Empty DataFrame
Columns: [Expiration Date, Long Strike, Short Strike, Required_Contracts, Credit, Total Premo, Max Loss, EV, Win Prob]
Index: []

In [26]:
call_spreads_list = []
for idx, row in weekly_calls.iterrows():
    call_spreads = weekly_calls[weekly_calls.Strike > row.Strike][['Expiration Date','Strike','Ask','Delta']]
    if len(call_spreads) < 3:
        continue
    else:
        call_spreads.columns = ['Expiration Date','Long Strike','Long Ask', 'Long Delta']
        call_spreads['Short Strike'] = row.Strike
        call_spreads['Short Bid'] = row.Bid
        call_spreads['Short Delta'] = row.Delta
        call_spreads['Weekly IV'] = (vix.reset_index().loc[len(vix) - 1, '^VIX_close']/100)*np.sqrt(time_remaining)
    call_spreads_list.append(create_spreads(call_spreads, 'calls', net_delta_threshold))

call_spreads = pd.concat(call_spreads_list, axis = 0).dropna().reset_index(drop = True)
call_spreads = call_spreads[(call_spreads['Total Premo'] > weekly_calls.loc[0,'Bid']*100) & 
                            (call_spreads['Prob of Better Than ES'] > 0.5) &
                            (call_spreads['Max Loss'] > -5000)].reset_index(drop = True)
call_spreads['Max Loss Differences'] = call_spreads['Max Loss'] - call_spreads['SPX Equiv Lower Loss']

call_spreads = call_spreads.sort_values('EV', ascending = False).reset_index(drop = True)
call_spreads[['Expiration Date','Long Strike','Short Strike','Required_Contracts','Credit','Total Premo','Max Loss','EV','Win Prob']]

Expiration Date  Long Strike  Short Strike  Required_Contracts    Credit  \
0       2019-02-20 2,740.000000  2,710.000000            2.000000 12.473400   
1       2019-02-20 2,735.000000  2,710.000000            3.000000 10.773400   
2       2019-02-20 2,745.000000  2,710.000000            2.000000 13.873400   
3       2019-02-20 2,730.000000  2,710.000000            3.000000  8.873400   
4       2019-02-20 2,740.000000  2,715.000000            3.000000  9.873400   
5       2019-02-20 2,725.000000  2,710.000000            4.000000  6.873400   
6       2019-02-20 2,750.000000  2,710.000000            2.000000 15.173400   
7       2019-02-20 2,735.000000  2,715.000000            3.000000  8.173400   
8       2019-02-20 2,745.000000  2,715.000000            2.000000 11.273400   
9       2019-02-20 2,730.000000  2,715.000000            4.000000  6.273400   
10      2019-02-20 2,725.000000  2,715.000000            6.000000  4.273400   
11      2019-02-20 2,720.000000  2,710.000000            6.000000  4.573400   
12      2019-02-20 2,750.000000  2,715.000000            2.000000 12.573400   
13      2019-02-20 2,740.000000  2,720.000000            3.000000  7.373400   
14      2019-02-20 2,735.000000  2,720.000000            4.000000  5.673400   
15      2019-02-20 2,740.000000  2,725.000000            4.000000  5.173400   
16      2019-02-20 2,745.000000  2,720.000000            3.000000  8.773400   
17      2019-02-20 2,735.000000  2,725.000000            6.000000  3.473400   
18      2019-02-20 2,730.000000  2,720.000000            6.000000  3.773400   
19      2019-02-20 2,720.000000  2,715.000000           13.000000  1.973400   
20      2019-02-20 2,725.000000  2,720.000000           13.000000  1.773400   
21      2019-02-20 2,715.000000  2,710.000000           13.000000  2.073400   
22      2019-02-20 2,730.000000  2,725.000000           13.000000  1.573400   
23      2019-02-20 2,740.000000  2,730.000000            7.000000  3.073400   
24      2019-02-20 2,755.000000  2,720.000000            2.000000 11.073400   

    Total Premo      Max Loss        EV  Win Prob  
0  2,494.680000 -3,505.320000  0.283464  0.578884  
1  3,232.020000 -4,267.980000  0.240975  0.568396  
2  2,774.680000 -4,225.320000  0.166086  0.587480  
3  2,662.020000 -3,337.980000  0.143488  0.556616  
4  2,962.020000 -4,537.980000  0.098882  0.593595  
5  2,749.360000 -3,250.640000  0.095115  0.544163  
6  3,034.680000 -4,965.320000  0.083220  0.595425  
7  2,452.020000 -3,547.980000  0.056369  0.583187  
8  2,254.680000 -3,745.320000 -0.018470  0.602117  
9  2,509.360000 -3,490.640000 -0.041137  0.571486  
10 2,564.040000 -3,435.960000 -0.089527  0.559101  
11 2,744.040000 -3,255.960000 -0.100771  0.529788  
12 2,514.680000 -4,485.320000 -0.101307  0.609987  
13 2,212.020000 -3,787.980000 -0.142057  0.608779  
14 2,269.360000 -3,730.640000 -0.184594  0.598470  
15 2,069.360000 -3,930.640000 -0.237931  0.625589  
16 2,632.020000 -4,867.980000 -0.259381  0.617212  
17 2,084.040000 -3,915.960000 -0.280495  0.615409  
18 2,264.040000 -3,735.960000 -0.282121  0.586867  
19 2,565.420000 -3,934.580000 -0.285425  0.544787  
20 2,305.420000 -4,194.580000 -0.330527  0.574572  
21 2,695.420000 -3,804.580000 -0.341775  0.514121  
22 2,045.420000 -4,454.580000 -0.378046  0.603937  
23 2,151.380000 -4,848.620000 -0.386276  0.642752  
24 2,214.680000 -4,785.320000 -0.596565  0.630944

In [5]:
stepsize = 0.05

weekly_puts['EV'] = np.nan
weekly_puts['Win Prob'] = np.nan

for idx, row in weekly_puts.iterrows():
    ev_df = pd.DataFrame({'SPX': np.arange(0, row['Strike'] + stepsize, stepsize)})
    ev_df = ev_df[(ev_df['SPX'] < row['Strike'] + stepsize)]
    ev_df['PnL'] = (row['Strike'] - ev_df['SPX'] - row.Ask)*100

    ev_df['Prob'] = norm.cdf(ev_df['SPX']/curr_spx - 1,0,
                             (vix.reset_index().loc[len(vix) - 1, '^VIX_close']/100)*np.sqrt(time_remaining))
    lb = ev_df.loc[0,'Prob']
    ub = ev_df.loc[len(ev_df) - 1,'Prob']

    ev_df['Prob'] = ev_df['Prob'].diff()
    ev_df.loc[0,'Prob'] = lb
    ev_df.loc[len(ev_df) - 1, 'Prob'] = 1 - ub

    ev_df['EV'] = ev_df.Prob*ev_df.PnL

    total_ev = sum(ev_df['EV'])
    win_prob = sum(ev_df[ev_df['PnL'] >= 0]['Prob'])
    
    weekly_puts.loc[idx, 'EV'] = total_ev
    weekly_puts.loc[idx, 'Win Prob'] = win_prob
    
weekly_puts

Expiration Date       Strike  Last Sale       Net       Bid       Ask  \
0       2019-02-13 2,705.000000  11.400000 -6.300000 11.200000 11.500000   
1       2019-02-13 2,700.000000   9.200000 -6.100000  9.300000  9.600000   
2       2019-02-13 2,695.000000   7.690000 -5.460000  7.600000  7.900000   
3       2019-02-13 2,690.000000   6.400000 -4.850000  6.300000  6.500000   
4       2019-02-13 2,685.000000   5.180000 -4.520000  5.100000  5.400000   
5       2019-02-13 2,680.000000   4.100000 -4.200000  4.200000  4.400000   
6       2019-02-13 2,675.000000   3.520000 -3.630000  3.400000  3.600000   
7       2019-02-13 2,670.000000   3.050000 -3.000000  2.800000  2.950000   
8       2019-02-13 2,665.000000   2.350000 -2.850000  2.300000  2.400000   
9       2019-02-13 2,660.000000   1.910000 -2.490000  1.850000  1.950000   
10      2019-02-13 2,655.000000   1.480000 -2.320000  1.500000  1.600000   
11      2019-02-13 2,650.000000   1.220000 -2.030000  1.250000  1.350000   
12      2019-02-13 2,645.000000   1.050000 -1.775000  1.000000  1.100000   
13      2019-02-13 2,640.000000   0.950000 -1.450000  0.800000  0.950000   
14      2019-02-13 2,635.000000   0.720000 -1.355000  0.650000  0.750000   
15      2019-02-13 2,630.000000   0.600000 -1.175000  0.550000  0.650000   

     Vol       IV     Delta    Gamma  Open Int         EV  Win Prob  
0    545 0.150800 -0.469900 0.012900       425 103.433126  0.345426  
1   1996 0.154700 -0.408200 0.012300      1939  69.603657  0.311770  
2    208 0.158800 -0.351600 0.011400       995  44.600009  0.278126  
3    267 0.162200 -0.299800 0.010500       945  17.267883  0.244034  
4    391 0.165100 -0.253000 0.009500       659 -14.079967  0.208918  
5    291 0.170500 -0.215000 0.008400      1760 -31.563684  0.176585  
6   1458 0.174300 -0.179900 0.007400       763 -47.590233  0.146252  
7    237 0.178400 -0.151000 0.006400       594 -59.746068  0.118748  
8    187 0.182000 -0.125100 0.005500       786 -65.663559  0.094617  
9    405 0.185900 -0.104100 0.004800      4273 -67.907717  0.073894  
10   201 0.190000 -0.086300 0.004100       594 -68.886716  0.056497  
11  1301 0.194300 -0.071800 0.003400      6411 -70.784756  0.042234  
12   221 0.198600 -0.059700 0.002900      1113 -65.521487  0.031036  
13   554 0.202100 -0.049100 0.002500      4575 -64.730269  0.022258  
14   504 0.206300 -0.040700 0.002100       432 -54.765886  0.015792  
15   204 0.212000 -0.034800 0.001800      1109 -51.717120  0.010838

In [6]:
weekly_calls['EV'] = np.nan
weekly_calls['Win Prob'] = np.nan

for idx, row in weekly_calls.iterrows():
    ev_df = pd.DataFrame({'SPX': np.arange(row['Strike'], curr_spx*1.20, stepsize)})
    ev_df['PnL'] = (ev_df['SPX'] - row['Strike'] - row.Ask)*100

    ev_df['Prob'] = norm.cdf(ev_df['SPX']/curr_spx - 1,0,
                             (vix.reset_index().loc[len(vix) - 1, '^VIX_close']/100)*np.sqrt(time_remaining))
    lb = ev_df.loc[0,'Prob']
    ub = ev_df.loc[len(ev_df) - 1,'Prob']

    ev_df['Prob'] = ev_df['Prob'].diff()
    ev_df.loc[0,'Prob'] = lb
    ev_df.loc[len(ev_df) - 1, 'Prob'] = 1 - ub

    ev_df['EV'] = ev_df.Prob*ev_df.PnL

    total_ev = sum(ev_df['EV'])
    win_prob = sum(ev_df[ev_df['PnL'] >= 0]['Prob'])
    
    weekly_calls.loc[idx, 'EV'] = total_ev
    weekly_calls.loc[idx, 'Win Prob'] = win_prob
    
weekly_calls

Expiration Date  Last Sale       Net       Bid       Ask   Vol       IV  \
0      2019-02-13  11.000000  1.700000 10.400000 10.700000  3950 0.146100   
1      2019-02-13   8.250000  0.900000  8.100000  8.400000   489 0.144600   
2      2019-02-13   6.000000  0.350000  6.100000  6.400000   380 0.141300   
3      2019-02-13   4.600000  0.300000  4.500000  4.700000   559 0.138900   
4      2019-02-13   3.300000  0.150000  3.200000  3.400000   845 0.136700   
5      2019-02-13   2.200000 -0.075000  2.200000  2.350000   550 0.134600   
6      2019-02-13   1.600000  0.025000  1.550000  1.600000  1348 0.134000   
7      2019-02-13   1.000000 -0.050000  1.000000  1.100000   297 0.133200   
8      2019-02-13   0.700000 -0.025000  0.600000  0.750000  1817 0.134100   

     Delta    Gamma  Open Int       Strike  time_remaining         EV  \
0 0.464100 0.013300       368 2,710.000000        0.006449 133.349560   
1 0.397500 0.013000       308 2,715.000000        0.005986 145.762909   
2 0.330600 0.012500       677 2,720.000000        0.005885 156.798891   
3 0.267100 0.011600      3859 2,725.000000        0.005825 165.174975   
4 0.209700 0.010300      1021 2,730.000000        0.005825 159.103508   
5 0.158200 0.008800       377 2,735.000000        0.005785 151.399245   
6 0.117700 0.007200       871 2,740.000000        0.005776 134.603475   
7 0.084500 0.005700       489 2,745.000000        0.005763 111.113163   
8 0.061400 0.004400      2407 2,750.000000        0.005776  88.304085   

   Win Prob  
0  0.342705  
1  0.313864  
2  0.283108  
3  0.251044  
4  0.217515  
5  0.184750  
6  0.153136  
7  0.123813  
8  0.097897

In [51]:
stepsize = 0.01

curr_spx = 2775.54
curr_vix = 14.89/100
t = 319/365.25
curr_iv = curr_vix*np.sqrt(t)

short_call_strike = 3025
long_call_strike = 3050
long_put_strike = 1725

number_of_cs = 3

credit = 13.45

ev_df = pd.DataFrame({'SPX': np.arange(0, long_call_strike + stepsize, stepsize)})

ev_df['Put_PnL'] = long_put_strike - ev_df.SPX
ev_df.loc[ev_df['Put_PnL']<0,'Put_PnL'] = 0

ev_df['Long_Call_PnL'] = number_of_cs*(ev_df.SPX - long_call_strike)
ev_df.loc[ev_df['Long_Call_PnL']<0,'Long_Call_PnL'] = 0

ev_df['Short_Call_PnL'] = number_of_cs*(ev_df.SPX - short_call_strike)
ev_df.loc[ev_df['Short_Call_PnL']<0,'Short_Call_PnL'] = 0

ev_df['PnL'] = ev_df.Put_PnL + ev_df.Long_Call_PnL - ev_df.Short_Call_PnL + credit

ev_df['Prob'] = norm.cdf(ev_df['SPX']/curr_spx - 1,0, curr_iv)
lb = ev_df.loc[0,'Prob']
ub = ev_df.loc[len(ev_df) - 1,'Prob']
ev_df['Prob'] = ev_df['Prob'].diff()
ev_df.loc[0,'Prob'] = lb
ev_df.loc[len(ev_df) - 1, 'Prob'] = 1 - ub
ev_df['EV'] = ev_df.Prob*ev_df.PnL

total_ev = sum(ev_df['EV'])
win_prob = sum(ev_df[ev_df['PnL'] >= 0]['Prob'])

conditional_win = ev_df[ev_df['PnL'] >= 0]
conditional_win['Cond_Prob'] = conditional_win.Prob/sum(conditional_win.Prob)


conditional_loss = ev_df[ev_df['PnL'] < 0]
conditional_loss['Cond_Prob'] = conditional_loss.Prob/sum(conditional_loss.Prob)


print(total_ev)
print(win_prob)
print(sum(conditional_win.Cond_Prob*conditional_win.PnL))
print(sum(conditional_loss.Cond_Prob*conditional_loss.PnL))

-4.82959586073
0.744566829578
13.9311562231
-59.5175256765


In [52]:
ev_df

SPX      Put_PnL  Long_Call_PnL  Short_Call_PnL          PnL  \
0          0.000000 1,725.000000       0.000000        0.000000 1,738.450000   
1          0.010000 1,724.990000       0.000000        0.000000 1,738.440000   
2          0.020000 1,724.980000       0.000000        0.000000 1,738.430000   
3          0.030000 1,724.970000       0.000000        0.000000 1,738.420000   
4          0.040000 1,724.960000       0.000000        0.000000 1,738.410000   
5          0.050000 1,724.950000       0.000000        0.000000 1,738.400000   
6          0.060000 1,724.940000       0.000000        0.000000 1,738.390000   
7          0.070000 1,724.930000       0.000000        0.000000 1,738.380000   
8          0.080000 1,724.920000       0.000000        0.000000 1,738.370000   
9          0.090000 1,724.910000       0.000000        0.000000 1,738.360000   
10         0.100000 1,724.900000       0.000000        0.000000 1,738.350000   
11         0.110000 1,724.890000       0.000000        0.000000 1,738.340000   
12         0.120000 1,724.880000       0.000000        0.000000 1,738.330000   
13         0.130000 1,724.870000       0.000000        0.000000 1,738.320000   
14         0.140000 1,724.860000       0.000000        0.000000 1,738.310000   
15         0.150000 1,724.850000       0.000000        0.000000 1,738.300000   
16         0.160000 1,724.840000       0.000000        0.000000 1,738.290000   
17         0.170000 1,724.830000       0.000000        0.000000 1,738.280000   
18         0.180000 1,724.820000       0.000000        0.000000 1,738.270000   
19         0.190000 1,724.810000       0.000000        0.000000 1,738.260000   
20         0.200000 1,724.800000       0.000000        0.000000 1,738.250000   
21         0.210000 1,724.790000       0.000000        0.000000 1,738.240000   
22         0.220000 1,724.780000       0.000000        0.000000 1,738.230000   
23         0.230000 1,724.770000       0.000000        0.000000 1,738.220000   
24         0.240000 1,724.760000       0.000000        0.000000 1,738.210000   
25         0.250000 1,724.750000       0.000000        0.000000 1,738.200000   
26         0.260000 1,724.740000       0.000000        0.000000 1,738.190000   
27         0.270000 1,724.730000       0.000000        0.000000 1,738.180000   
28         0.280000 1,724.720000       0.000000        0.000000 1,738.170000   
29         0.290000 1,724.710000       0.000000        0.000000 1,738.160000   
...             ...          ...            ...             ...          ...   
304971 3,049.710000     0.000000       0.000000       74.130000   -60.680000   
304972 3,049.720000     0.000000       0.000000       74.160000   -60.710000   
304973 3,049.730000     0.000000       0.000000       74.190000   -60.740000   
304974 3,049.740000     0.000000       0.000000       74.220000   -60.770000   
304975 3,049.750000     0.000000       0.000000       74.250000   -60.800000   
304976 3,049.760000     0.000000       0.000000       74.280000   -60.830000   
304977 3,049.770000     0.000000       0.000000       74.310000   -60.860000   
304978 3,049.780000     0.000000       0.000000       74.340000   -60.890000   
304979 3,049.790000     0.000000       0.000000       74.370000   -60.920000   
304980 3,049.800000     0.000000       0.000000       74.400000   -60.950000   
304981 3,049.810000     0.000000       0.000000       74.430000   -60.980000   
304982 3,049.820000     0.000000       0.000000       74.460000   -61.010000   
304983 3,049.830000     0.000000       0.000000       74.490000   -61.040000   
304984 3,049.840000     0.000000       0.000000       74.520000   -61.070000   
304985 3,049.850000     0.000000       0.000000       74.550000   -61.100000   
304986 3,049.860000     0.000000       0.000000       74.580000   -61.130000   
304987 3,049.870000     0.000000       0.000000       74.610000   -61.160000   
304988 3,049.880000     0.000000       0.000000       74.640000   -61.190000   
304989 3,049.890